## Connection to a SQL database

In [ ]:
import os
import mysql.connector

# Access environment variable directly (may raise KeyError if not set)
database_name = os.environ['MYSQL_DATABASE']
user = os.environ['MYSQL_USER']
password = os.environ['MYSQL_PASSWORD']

mydb = mysql.connector.connect(
  host="mysql",  # Use service name from docker-compose
  user=user,
  password=password,
  database=database_name
)

mycursor = mydb.cursor()

In [3]:
# Define your table schema
sql = """CREATE TABLE IF NOT EXISTS users (
  id INT AUTO_INCREMENT PRIMARY KEY,
  username VARCHAR(255) NOT NULL,
  email VARCHAR(255) NOT NULL UNIQUE
)"""

mycursor.execute(sql)

In [4]:
sql = "INSERT INTO users (username, email) VALUES (%s, %s)"
val = ("John Doe", "johndoe@example.com")
mycursor.execute(sql, val)

# Insert multiple rows at once using executemany
sql = "INSERT INTO users (username, email) VALUES (%s, %s)"
val = [
  ("Jane Doe", "janedoe@example.com"),
  ("Peter Smith", "petersmith@example.com"),
]
mycursor.executemany(sql, val)

mydb.commit()  # Commit changes to the database

## Conecction to a Virtuoso Triple Store

In [1]:
from rdflib.graph import ConjunctiveGraph as Graph
from rdflib.store import Store
from rdflib.plugin import get as plugin
from rdflib.term import URIRef, Literal

# DSN with connection details
# Virtuoso connection details
username = "dba"
password = "dba"
host = "virtuoso"
port = 8890
dsn = "VOS;UID=%s;PWD=%s;HOST=%s;PORT=%s" % (username, password, host, port) 

# Create Virtuoso store object
Virtuoso = plugin("Virtuoso", Store)
graph = Graph(Virtuoso(dsn))

PluginException: No plugin registered for (Virtuoso, <class 'rdflib.store.Store'>)

In [ ]:
# Define resources
subject = URIRef("http://example.com/resource1")
predicate = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type")
object = URIRef("http://example.com/ClassA")

# Add triple to the graph
graph.add((subject, predicate, object))

# Commit changes to Virtuoso
graph.commit()

## Connection to a Anzograph DB

### Aproach 1

In [61]:
import rdflib.plugins.stores.sparqlstore as sprql_store
from rdflib import Namespace ,Graph, URIRef, Literal, BNode, store
from rdflib.namespace import FOAF, RDF
from rdflib.plugins.stores import sparqlstore
from rdflib.graph import DATASET_DEFAULT_GRAPH_ID as default


store = sparqlstore.SPARQLUpdateStore()
query_endpoint = 'http://anzograph:7070/sparql'
update_endpoint = 'http://anzograph:7070/sparql'
store.open((query_endpoint, update_endpoint))

# Define example namespace.
ex = Namespace("http://example_asdsfdas.org/")

# Define a node.
node = (ex.me, RDF.type, FOAF.Person)

# Open a graph in the open store and set identifier to default graph ID.
graph = Graph(store, identifier="http://example.com/graph1")

# Add node to graph.
graph.add(node)

<Graph identifier=http://example.com/graph1 (<class 'rdflib.graph.Graph'>)>

In [64]:
result = graph.query(""" SELECT ?s (COUNT(*) as ?count)
WHERE {
		?s ?p ?o
}
GROUP BY ?s
ORDER BY DESC(?count)""")
print("Predicates:  ")
for row in result:
    print(row)


result = g.query(""" SELECT ?g (COUNT(*) as ?count)
WHERE {
	graph ?g{
		?s ?p ?o
	}
}
GROUP BY ?g
ORDER BY DESC(?count)""")
print("Graphs:  ")
for row in result:
    print(row)

Predicates:  
(rdflib.term.URIRef('http://example.com/my#NewObject'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))
(rdflib.term.URIRef('http://example.com/my#NewObject2'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))
(rdflib.term.URIRef('http://example.com/my#NewObject1'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))
(rdflib.term.URIRef('http://example_asdsfdas.org/me'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))
Graphs:  
(rdflib.term.URIRef('http://example.com/graph1'), rdflib.term.Literal('4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))


In [67]:
graph3 = Graph(store, identifier="http://example.com/graph3")

graph3.parse(
    data="""
        <x:> a <c:> .
        <y:> a <c:> .
    """,
    format="turtle"
)

# Select all the things (s) that are of type (rdf:type) c:
qres = graph3.query("""SELECT ?s WHERE { ?s a <c:> }""")

for row in qres:
    print(f"{row.s}")
# prints:
# x:
# y:

# Add in a new triple using SPARQL UPDATE
graph3.update("""INSERT DATA { <z:> a <c:> }""")

# Select all the things (s) that are of type (rdf:type) c:
qres = graph3.query("""SELECT ?s WHERE { ?s a <c:> }""")

print("After update:")
for row in qres:
    print(f"{row.s}")
# prints:
# x:
# y:
# z:

# Change type of <y:> from <c:> to <d:>
graph3.update("""
         DELETE { <y:> a <c:> }
         INSERT { <y:> a <d:> }
         WHERE { <y:> a <c:> }
         """)
print("After second update:")
qres = graph3.query("""SELECT ?s ?o WHERE { ?s a ?o }""")
for row in qres:
    print(f"{row.s} a {row.o}")
# prints:
# x: a c:
# z: a c:
# y: a d:

x:
y:
After update:
x:
z:
y:
After second update:
x: a c:
z: a c:
y: a d:


### Aproach 2

In [59]:
import rdflib
import rdflib.plugins.stores.sparqlstore as sprql_store
from rdflib import Graph, URIRef, Literal, BNode, store
from rdflib.namespace import FOAF, RDF

store = sprql_store.SPARQLStore("http://anzograph:7070/sparql")
g = Graph(store=store)

result = g.query(""" SELECT ?s (COUNT(*) as ?count)
WHERE {
		?s ?p ?o
}
GROUP BY ?s
ORDER BY DESC(?count)""")
print("Predicates:  ")
for row in result:
    print(row)

result = g.query(""" SELECT ?g (COUNT(*) as ?count)
WHERE {
	graph ?g{
		?s ?p ?o
	}
}
GROUP BY ?g
ORDER BY DESC(?count)""")
print("Graphs:  ")
for row in result:
    print(row)

Predicates:  
(rdflib.term.URIRef('http://example.org/me'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))
(rdflib.term.URIRef('http://example.com/book1'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))
(rdflib.term.URIRef('http://example_asdsfdas.org/me'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))
(rdflib.term.URIRef('x:'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))
(rdflib.term.URIRef('y:'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))
Graphs:  
(rdflib.term.URIRef('http://example.com/graph1'), rdflib.term.Literal('4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#long')))


In [55]:
g.parse(
    data="""
        <x:> a <c:> .
        <y:> a <c:> .
    """,
    format="turtle"
)

# Select all the things (s) that are of type (rdf:type) c:
qres = g.query("""SELECT ?s WHERE { ?s a <c:> }""")

for row in qres:
    print(f"{row.s}")
# prints:
# x:
# y:

# Add in a new triple using SPARQL UPDATE
g.update("""INSERT DATA { <z:> a <c:> }""")

# Select all the things (s) that are of type (rdf:type) c:
qres = g.query("""SELECT ?s WHERE { ?s a <c:> }""")

print("After update:")
for row in qres:
    print(f"{row.s}")
# prints:
# x:
# y:
# z:

# Change type of <y:> from <c:> to <d:>
g.update("""
         DELETE { <y:> a <c:> }
         INSERT { <y:> a <d:> }
         WHERE { <y:> a <c:> }
         """)
print("After second update:")
qres = g.query("""SELECT ?s ?o WHERE { ?s a ?o }""")
for row in qres:
    print(f"{row.s} a {row.o}")
# prints:
# x: a c:
# z: a c:
# y: a d:

HTTPError: HTTP Error 400: Bad Request

### Aproach 3

In [27]:
import azg3

result = azg3.create_dataframe("anzograph:7070" , """ SELECT ?g (COUNT(*) as ?count)
WHERE {
	graph ?g{
		?s ?p ?o
	}
}
GROUP BY ?g
ORDER BY DESC(?count)""")

print(result)

                             g  count
0  <http://example.com/graph1>      3


In [26]:
result = azg3.run_query("anzograph:7070" , """ 
PREFIX my: <http://example.com/my#>  # Replace with your actual prefix

INSERT DATA {
  GRAPH <http://example.com/graph1> {
    my:NewObject1 my:newProperty "New Value 1" .
    my:NewObject2 my:anotherProperty "Another Value 2" .
  }
};
""")

print (result)

b''


In [22]:
result = azg3.run_query("anzograph:7070" , """ 
INSERT DATA {
  <http://example.com/book1> <http://purl.org/dc/elements/1.1/title> "Introduction to SPARQL" .
}
"""                          
)
print(result)

b''


In [33]:
result = azg3.create_dataframe("anzograph:7070" , """ SELECT ?s (COUNT(*) as ?count)
WHERE {
		?s ?p ?o
}
GROUP BY ?s
ORDER BY DESC(?count)""")

print(result)


                                  s  count
0  <http://example_asdsfdas.org/me>      1
1        <http://example.com/book1>      1
2           <http://example.org/me>      1


In [ ]:
result = azg3.create_dataframe("anzograph:7070" , """ SELECT ?s (COUNT(*) as ?count)
WHERE {
		?s ?p ?o
}
GROUP BY ?s
ORDER BY DESC(?count)""")

print(result)